# Making a movie of reaction-diffusion concentrations

We recommend creating and using a virtual environment for NetPyNE tutorials.  To do so, enter the following commands into your terminal:

    mkdir netpyne_tuts
    cd netpyne_tuts
    python3 -m venv env
    source env/bin/activate
    python3 -m pip install --upgrade pip wheel ipython ipykernel jupyter
    python3 -m pip install --upgrade neuron
    git clone https://github.com/Neurosim-lab/netpyne.git
    python3 -m pip install -e netpyne
    ipython kernel install --user --name=env
    
For this tutorial, you will also need to install `natsort` and `imageio`.

    python3 -m pip install natsort imageio

Then you can copy this Jupyter notebook and the example directory we will be using into `netpyne_tuts` and compile the mod files.
    
    cp netpyne/netpyne/tutorials/rxd_movie_tut/rxd_movie_tut.ipynb .
    cp netpyne/examples/rxd_net .
    cd rxd_net
    nrnivmodl mod
    
Finally, you can launch this tutorial in a Jupyter notebook.

    jupyter notebook rxd_movie_tut.ipynb

In [ ]:
import os
import natsort
import imageio

images = []
filenames = natsort.natsorted([file for file in os.listdir() if 'movie' in file and file.endswith('.png')])
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('rxd_conc_movie.gif', images)

Your movie should show up below:

<img src="rxd_conc_movie.gif">